# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 61 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.04136


/tmp/ipykernel_2089/4030337529.py:34: LatexWarning: 2301.04136 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2301.04145


extracting tarball to tmp_2301.04145...

...

extracting tarball to tmp_2301.04586...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.04637


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.04586/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2301.04637...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04442-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04442) | **The CARMENES search for exoplanets around M dwarfs. A long-period planet  around GJ 1151 measured with CARMENES and HARPS-N data**  |
|| J. Blanco-Pozo, et al. -- incl., <mark>M. Pérez-Torres</mark>, <mark>E. L. Brown</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *18 pages, 11 figures. Accepted version, A&A (2023)*|
|**Abstract**| Detecting a planetary companion in a short-period orbit through radio emission from the interaction with its host star is a new prospect in exoplanet science. Recently, a tantalising signal was found close to the low-mass stellar system GJ 1151 using LOFAR observations. We studied spectroscopic time-series data of GJ 1151 in order to search for planetary companions, investigate possible signatures of stellar magnetic activity, and to find possible explanations for the radio signal. We used the combined radial velocities measured from spectra acquired with the CARMENES, HARPS-N, and HPF instruments, extracted activity indices from those spectra in order to mitigate the impact of stellar magnetic activity on the data, and performed a detailed analysis of Gaia astrometry and all available photometric time series coming from the MEarth and ASAS-SN surveys. We found a M$>$10.6 M$_{\oplus}$ companion to GJ 1151 in a 390d orbit at a separation of 0.57 au. Evidence for a second modulation is also present; this could be due to long-term magnetic variability or a second (substellar) companion. The star shows episodes of elevated magnetic activity, one of which could be linked to the observed LOFAR radio emission. We show that it is highly unlikely that the detected GJ 1151 b, or any additional outer companion is the source of the detected signal. We cannot firmly rule out the suggested explanation of an undetected short-period planet that could be related to the radio emission, as we establish an upper limit of 1.2 M$_{\oplus}$ for the minimum mass. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04533-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04533) | **Very long baseline interferometry observations of the high-redshift  blazar candidate J0141-5427**  |
|| K. É. Gabányi, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *8 pages, 4 figures, accepted for publication in PASA*|
|**Abstract**| Active galactic nuclei (AGN) have been observed as far as redshift z~7. They are crucial in investigating the early Universe as well as the growth of supermassive black holes at their centres. Radio-loud AGN with their jets seen at a small viewing angle are called blazars and show relativistic boosting of their emission. Thus, their apparently brighter jets are easier to detect in the high-redshift Universe. DES J014132.4-542749.9 is a radio-luminous but X-ray weak blazar candidate at z = 5. We conducted high-resolution radio interferometric observations of this source with the Australian Long Baseline Array at 1.7 and 8.5 GHz. A single, compact radio emitting feature was detected at both frequencies with a flat radio spectrum. We derived the milliarcsecond-level accurate position of the object. The frequency dependence of its brightness temperature is similar to that of blazar sources observed at lower redshifts. Based on our observations, we can confirm its blazar nature. We compared its radio properties with those of two other similarly X-ray-weak and radio-bright AGN, and found that they show very different relativistic boosting characteristics. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04163-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04163) | **Gamma-ray emission from spectrally resolved cosmic rays in galaxies**  |
|| Maria Werhahn, Philipp Girichidis, Christoph Pfrommer, <mark>Joseph Whittingham</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *18 pages, 11 figures, submitted to MNRAS, comments are welcome!*|
|**Abstract**| Cosmic rays (CRs) are ubiquitous in the interstellar medium (ISM) of nearby galaxies, but many of their properties are not well-constrained. Gamma-ray observations provide a powerful tool in this respect, allowing us to constrain both the interaction of CR protons with the ISM and their transport properties. To help better understand the link between observational signatures and CR physics, we use a series of magneto-hydrodynamical (MHD) AREPO simulations of isolated galaxies performed using spectrally-resolved CR transport in every computational cell, with subsequent gamma-ray emission calculated using the CRAYON+ (Cosmic RAY emissiON) code. In each of our simulated halos, modelling the energy-dependent spatial diffusion of CRs leads to a more extended distribution of high-energy (~100 GeV) gamma rays compared to that predicted by a 'grey' steady-state model, which is especially visible in the corresponding emission maps and radial profiles. Despite this, the total gamma-ray spectra can often be well approximated by the steady-state model, although recovering the same spectral index typically requires a minor variation of the energy dependence of the diffusion coefficient. Our simulations reproduce the observed spectral indices and gamma-ray spectra of nearby star-forming galaxies and also match recent observations of the far infrared--gamma-ray relation. We find, however, that the spectrally resolved model yields marginally smaller luminosities for lower star formation rates compared to grey simulations of CRs. Our work highlights the importance of modelling spectrally resolved CR transport for an accurate prediction of spatially resolved high-energy gamma-ray emission, as will be probed by the upcoming Cherenkov Telescope Array observatory. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04637-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04637) | **A Systematic Study of Ia-CSM Supernovae from the ZTF Bright Transient  Survey**  |
|| <mark>Yashvi Sharma</mark>, et al. -- incl., <mark>Nora Linn Strotjohann</mark>, <mark>Frank J. Masci</mark>, <mark>Lin Yan</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *26 pages, 14 figures, 5 tables. Submitted to ApJ*|
|**Abstract**| Among the supernovae (SNe) that show strong interaction with the circumstellar medium, there is a rare subclass of Type Ia supernovae, SNe Ia-CSM, that show strong narrow hydrogen emission lines much like SNe IIn but on top of a diluted over-luminous Type Ia spectrum. In the only previous systematic study of this class (Silverman et al. 2013), 16 objects were identified, 8 historic and 8 from the Palomar Transient Factory (PTF). Now using the successor survey to PTF, the Zwicky Transient Facility (ZTF), we have classified 12 additional objects of this type through the systematic Bright Transient Survey (BTS). In this study, we present and analyze the optical and mid-IR light curves, optical spectra, and host galaxy properties of this sample. Consistent with previous studies, we find the objects to have slowly evolving light curves compared to normal SNe Ia with peak absolute magnitudes between -19.1 and -21, spectra having weak H$\beta$, large Balmer decrements of ~7 and strong Ca NIR emission. Out of 10 SNe from our sample observed by NEOWISE, 9 have $3\sigma$ detections, along with some showing a clear reduction in red-wing of H$\alpha$, indicative of newly formed dust. We do not find our SN Ia-CSM sample to have a significantly different distribution of equivalent width of He I $\lambda5876$ than SNe IIn as observed in Silverman et al. 2013. The hosts tend to be late-type galaxies with recent star formation. We also derive a rate estimate of 29$^{+27}_{-21}$ Gpc$^{-3}$ yr$^{-1}$ for SNe Ia-CSM which is ~0.02--0.2 % of the SN Ia rate. This work nearly doubles the sample of well-studied Ia-CSM objects in Silverman et al. 2013, increasing the total number to 28. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04498-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04498) | **VTSCat: The VERITAS Catalog of Gamma-Ray Observations**  |
|| W. Benbow, et al. -- incl., <mark>A. Brill</mark>, <mark>A. Gent</mark>, <mark>O. Hervet</mark>, <mark>G. Hughes</mark>, <mark>T. K. Kleiner</mark>, <mark>M. J. Lang</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| **|
|**Abstract**| The ground-based gamma-ray observatory VERITAS (Very Energetic Radiation Imaging Telescope Array System) is sensitive to photons of astrophysical origin with energies in the range between $\approx 85$ GeV to $\approx 30$ TeV. The instrument consists of four 12-m diameter imaging Cherenkov telescopes operating at the Fred Lawrence Whipple Observatory (FLWO) in southern Arizona. VERITAS started four-telescope operations in 2007 and collects about 1100 hours of good-weather data per year. The VERITAS collaboration has published over 100 journal articles since 2008 reporting on gamma-ray observations of a large variety of objects: Galactic sources like supernova remnants, pulsar wind nebulae, and binary systems; extragalactic sources like star forming galaxies, dwarf-spheroidal galaxies, and highly-variable active galactic nuclei. This note presents VTSCat: the catalog of high-level data products from all VERITAS publications. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04409-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04409) | **Comprehensive spectroscopic and photometric study of pulsating eclipsing  binary star AI Hya**  |
|| F. Kahraman Alivacus, et al. -- incl., <mark>K. G. Hełminiak</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| The pulsating eclipsing binaries are remarkable systems that provide an opportunity to probe the stellar interior and to determine the fundamental stellar parameters precisely. Especially the detached eclipsing binary systems with (a) pulsating component(s) are significant objects to understand the nature of the oscillations since the binary effects in these systems are negligible. Recent studies based on space data have shown that the pulsation mechanisms of some oscillating stars are not completely understood. Hence, comprehensive studies of a number of pulsating stars within detached eclipsing binaries are important. In this study, we present a detailed analysis of the pulsating detached eclipsing binary system AI Hya which was studied by two independent groups with different methods. We carried out a spectroscopic survey to estimate the orbital parameters via radial velocity measurements and the atmospheric parameters of each binary component using the composite and/or disentangled spectra. We found that the more luminous component of the system is a massive, cool and chemically normal star while the hotter binary component is a slightly metal-rich object. The fundamental parameters of AI Hya were determined by the analysis of binary variations and subsequently used in the evolutionary modelling. Consequently, we obtained the age of the system as 850 $\pm$ 20 Myr and found that both binary components are situated in the Delta Scuti instability strip. The frequency analysis revealed pulsation frequencies between the 5.5 - 13.0 d$^{-1}$ and we tried to estimate which binary component is the pulsating one. However, it turned out that those frequencies could originate from both binary components. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xc7 in position 3324: invalid continuation byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04136-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04136) | **Focused Space Weather Strategy for Securing Earth, and Human Exploration  of the Moon and Mars**  |
|| A. Posner, et al. -- incl., <mark>B. Heber</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| *Heliophysics 2050 White Paper*|
|**Abstract**| This white paper recognizes gaps in observations that will, when addressed, much improve solar radiation hazard and geomagnetic storm forecasting. Radiation forecasting depends on observations of the entire "Solar Radiation Hemisphere" that we will define. Mars exploration needs strategic placement of radiation-relevant observations. We also suggest an orbital solution that will improve geomagnetic storm forecasting through improved in situ and solar/heliospheric remote sensing. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.04476-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.04476) | **Mid-infrared bi-directional reflectance spectroscopy of impact melt  glasses and tektites**  |
|| Andreas Morlok, et al. -- incl., <mark>Joern Helbert</mark> |
|*Appeared on*| *2023-01-12*|
|*Comments*| **|
|**Abstract**| We have analyzed 14 impact melt glass samples, covering the compositional range from highly felsic to mafic/basaltic, as part of our effort to provide mid-infrared spectra (7-14 micron) for MERTIS (Mercury Radiometer and Thermal Infrared Spectrometer), an instrument onboard of the ESA/JAXA BepiColombo mission. Since Mercury was exposed to many impacts in its history, and impact glasses are also common on other bodies, powders of tektites (Irghizite, Libyan Desert Glass, Moldavite, Muong Nong, Thailandite) and impact glasses (from the Dellen, El'gygytgyn, Lonar, Mien, Mistastin, and Popigai impact structures) were analyzed in four size fractions of (0-25, 25-63, 93-125 and 125-250 micron) from 2.5 to 19 micron in bi-directional reflectance. The characteristic Christiansen Feature (CF) is identified between 7.3 micron (Libyan Desert Glass) and 8.2 micron (Dellen). Most samples show mid-infrared spectra typical of highly amorphous material, dominated by a strong Reststrahlen Band (RB) between 8.9 micron (Libyan Desert Glass) and 10.3 micron (Dellen). Even substantial amounts of mineral fragments hardly affect this general band shape. Comparisons of the SiO2 content representing the felsic/mafic composition of the samples with the CF shows felsic/intermediate glass and tektites forming a big group, and comparatively mafic samples a second one. An additional sign of a highly amorphous state is the lack of features at wavelengths longer than about 15 micron. The tektites and two impact glasses, Irghizite and El'gygytgyn respectively, have much weaker water features than most of the other impact glasses. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.04442.md
    + _build/html/tmp_2301.04442/./pics/selection/Sin_Quad_LS_data_3_subplot.png
    + _build/html/tmp_2301.04442/./pics/selection/p1_quad_res_quad_term_data_LS_no_label.png
    + _build/html/tmp_2301.04442/./pics/selection/p1_quad_res_data_LS_no_label.png
    + _build/html/tmp_2301.04442/./pics/selection/LS_photometry_MEarth_individually.jpeg
    + _build/html/tmp_2301.04442/./pics/selection/LS_photometry_selection.jpeg
    + _build/html/tmp_2301.04442/./pics/selection/significant_indices_LS_1409.jpeg
exported in  _build/html/2301.04533.md
    + _build/html/tmp_2301.04533/./J0141_L.png
    + _build/html/tmp_2301.04533/./J0141_Xamplcal.png
    + _build/html/tmp_2301.04533/./radiospectrum.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arraystretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arraystretch$}{1.0}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.04442-b31b1b.svg)](https://arxiv.org/abs/2301.04442)<mark>Appeared on: 2023-01-12</mark> - _18 pages, 11 figures. Accepted version, A&A (2023)_

</div>
<div id="authors">

J. Blanco-Pozo, et al. -- incl., <mark><mark>M. Pérez-Torres</mark></mark>, <mark><mark>E. L. Brown</mark></mark>, <mark><mark>M. Kürster</mark></mark>

</div>
<div id="abstract">

**Abstract:** Detecting a planetary companion in a short-period orbit through radio emission from the interaction with its host star is a new prospect in exoplanet science. Recently, a tantalising signal was found  close to the low-mass stellar system GJ 1151 using LOFAR observations.We studied spectroscopic time-series data of GJ 1151 in order to search for planetary companions, investigate possible signatures of stellar magnetic activity, and to find possible explanations for the radio signal.We used the combined radial velocities measured from spectra acquired with the CARMENES, HARPS-N, and HPF instruments, extracted activity indices from those spectra in order to mitigate the impact of stellar magnetic activity on the data, and performed a detailed analysis of{\it Gaia}astrometry and all available photometric time series coming from the MEarth and ASAS-SN surveys.We found a M$>$10.6 M$_{\oplus}$companion to GJ 1151 in a 390d orbit at a separation of 0.57 au. Evidence for a second modulation is also present; this could be due to long-term magnetic variability or a second (substellar) companion. The star shows episodes of elevated magnetic activity, one of which could be linked to the observed LOFAR radio emission. We show that it is highly unlikely that the detected GJ 1151 b, or any additional outer companion is the source of the detected signal. We cannot firmly rule out the suggested explanation of an undetected short-period planet that could be relatedto the radio emission,  as we establish an upper limit of 1.2 M$_{\oplus}$for the minimum mass.

</div>

<div id="div_fig1">

<img src="tmp_2301.04442/./pics/selection/Sin_Quad_LS_data_3_subplot.png" alt="Fig7.1" width="33%"/><img src="tmp_2301.04442/./pics/selection/p1_quad_res_quad_term_data_LS_no_label.png" alt="Fig7.2" width="33%"/><img src="tmp_2301.04442/./pics/selection/p1_quad_res_data_LS_no_label.png" alt="Fig7.3" width="33%"/>

**Figure 7. -** RV time-series data (left panels) and periodograms (red dashed lines as the 0.1, 1, and 10\% FAP levels) of the best-fitting model consisting of a sinusoid and a parabola. The middle panels show the low-frequency portion of the periodograms ($P>50$ d) and we mark the rotational period at 140,  including its uncertainty of $\pm$10 d, with the blue vertical area, and the planetary period at 390 d with the green vertical line. In the right panel, the full periodogram is shown. In the top panels, we show the data as observed (green: CARMENES, red: HPF, blue: HARPS-N) and indicate the best-fitting model by the black curve. The middle row shows the RV residuals after the subtraction of the parabola. The bottom row shows the residuals of the best fit and we provide the RV rms. (*fig:best_models1*)

</div>
<div id="div_fig2">

<img src="tmp_2301.04442/./pics/selection/LS_photometry_MEarth_individually.jpeg" alt="Fig8" width="100%"/>

**Figure 8. -** Individual MEarth datasets separated according to observational season and telescope (left panels). We also show  their periodograms for longer periods ($>$50 days, middle panels) and the full periodograms (right panels). We mark the significant signals in grey, the period of our planet in green, and the rotational period of the star in blue, within the errors. The FAP levels of 10, 1, and 0.1\% are shown by the red dashed lines. (*prewhit1*)

</div>
<div id="div_fig3">

<img src="tmp_2301.04442/./pics/selection/LS_photometry_selection.jpeg" alt="Fig6.1" width="50%"/><img src="tmp_2301.04442/./pics/selection/significant_indices_LS_1409.jpeg" alt="Fig6.2" width="50%"/>

**Figure 6. -** Photometric and spectral index time series. Significant signals are marked. We show the full dataset (left panels) and their periodograms for longer periods ($>$50 d, middle panels) and for all periods (right panels). The grey vertical dashed lines mark the significant peaks. We show the period of the planet  in green and the rotational period of the star at 140$\pm$10 d  in blue. For the combined MEarth data, we show the different individual sets in colours. The red dashed horizontal lines show the FAP levels of 10, 1, and 0.1\%. (*LS_indices*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\}{natexlab}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# 5427

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.04533-b31b1b.svg)](https://arxiv.org/abs/2301.04533)<mark>Appeared on: 2023-01-12</mark> - _8 pages, 4 figures, accepted for publication in PASA_

</div>
<div id="authors">

K. \'E.~Gab\'anyi, et al. -- incl., <mark>S.~Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** Active galactic nuclei (AGN) have been observed as far as redshift$z\sim7$. They are crucial in investigating the early Universe as well as the growth of supermassive black holes at their centres. Radio-loud AGN with their jets seen at a small viewing angle are called blazars and show relativistic boosting of their emission. Thus, their apparently brighter jets are easier to detect in the high-redshift Universe.DES J014132.4$-$542749.9 is a radio-luminous but X-ray weak blazar candidate at$z=5$. We conducted high-resolution radio interferometric observations of this source with the Australian Long Baseline Array at$1.7$and$8.5$GHz. A single, compact radio emitting feature was detected at both frequencies with a flat radio spectrum. We derived the milliarcsecond-level accurate position of the object. The frequency dependence of its brightness temperature is similar to that of blazar sources observed at lower redshifts. Based on our observations, we can confirm its blazar nature. We compared its radio properties with those of two other similarly X-ray-weak and radio-bright AGN, and found that they show very different relativistic boosting characteristics.

</div>

<div id="div_fig1">

<img src="tmp_2301.04533/./J0141_L.png" alt="Fig1" width="100%"/>

**Figure 1. -** $1.7$-GHz naturally-weighted LBA map of the fringe-fitted data of J0141$-$5427. The peak intensity is $76.7\mathrm{ mJy beam}^{-1}$. The lowest contours are at $\pm1.7\mathrm{ mJy beam}^{-1}$, corresponding to $4\sigma$ image noise level. Further positive contours increase by a factor of $2$. The elliptical Gaussian restoring beam size is $25.7\mathrm{ mas}\times6.2\mathrm{ mas}$ at a major axis position angle of $-8.9^\circ$, and it is shown in the lower left corner of the image. (*fig:LBA_L*)

</div>
<div id="div_fig2">

<img src="tmp_2301.04533/./J0141_Xamplcal.png" alt="Fig2" width="100%"/>

**Figure 2. -** $8.5$-GHz naturally-weighted phase-referenced LBA map of J0141$-$5427. The peak intensity is $28.5\mathrm{ mJy beam}^{-1}$. The lowest contours are at $\pm0.7\mathrm{ mJy beam}^{-1}$, corresponding to $4\sigma$ image noise level. Further positive contours increase by a factor of $2$. The elliptical Gaussian restoring beam size is $3.9\mathrm{ mas}\times2.8\mathrm{ mas}$ at a major axis position angle of $5.3^\circ$, and it is shown in the lower left corner of the image. (*fig:LBA_X*)

</div>
<div id="div_fig3">

<img src="tmp_2301.04533/./radiospectrum.png" alt="Fig3" width="100%"/>

**Figure 3. -** Radio spectrum of J0141$-$5427. Black circles are low-resolution archival measurements \citep[for references, see][]{Belladitta_2019}. Orange circles are from the RACS DR1 \citep{racs1,racs2}, and from the SPT-SZ survey \citep{sptsz-cat}. Red squares are our LBA flux densities. The brown line represents a power-law fit to the low-resolution data (black and orange symbols). (*fig:spectrum*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

165  publications in the last 7 days.
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers